# Analyse de la qualité des données du fichier Clients_Master


## Résumé des anomalies observées
- 18 doublons (3%)
- 17% de villes non renseignées
<br> --> saisie manuelle facultative
- 61% des lignes comportent des villes qui n'existe pas ou son mal orthographiées (Pariss)
<br> --> saisie manuelle (faute de frappe, mauvais champs sélectionné)

## Proposition d'une correction
- supprimer les doublons (pas de perte de données)

In [9]:
import pandas as pd
import numpy as np
import requests

In [3]:
df_clients = pd.read_csv('../data/raw/Clients_Master.csv')

In [7]:
print("AUDIT DATAFRAME CLIENTS")

display(df_clients.head())
display(df_clients.info())

print("\n---Taux de valeurs manquantes (%)---")
missing_percentage = df_clients.isnull().sum() / len(df_clients) * 100
print(missing_percentage[missing_percentage > 0].sort_values(ascending=False))

print("\n---Vérification des doublons---")
clients_doublons = df_clients.duplicated().sum()
print(f"Nombre total de doublons: {clients_doublons} / soit {clients_doublons/len(df_clients) * 100:.2f}%")


AUDIT DATAFRAME CLIENTS


,client_id,nom,prenom,email,ville,date_inscription,derniere_activite
0,1,Rey,Alexandrie,alexandrie.rey@live.com,Lejeune,2023-10-15,2025-01-21
1,2,Lagarde,Eugène,eugène.lagarde@yahoo.fr,NaN,2024-07-31,2025-08-25
2,3,Gaudin,Aimée,aimée.gaudin@sfr.fr,Lopezdan,2024-01-03,2025-10-19
3,4,Faivre,Noël,noël.faivre@voila.fr,Saint Eugène,2023-08-01,2025-07-11
4,5,Faure,Véronique,véronique.faure@club-internet.fr,Sainte Jacques-les-Bains,2023-02-14,2025-10-23


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520 entries, 0 to 519
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   client_id          520 non-null    int64 
 1   nom                520 non-null    object
 2   prenom             520 non-null    object
 3   email              520 non-null    object
 4   ville              428 non-null    object
 5   date_inscription   520 non-null    object
 6   derniere_activite  520 non-null    object
dtypes: int64(1), object(6)
memory usage: 28.6+ KB


None


---Taux de valeurs manquantes (%)---
ville    17.692308
dtype: float64

---Vérification des doublons---
Nombre total de doublons: 18 / soit 3.46%


## On vérifie si les villes renseignées existent
## Le script est long (~1min30 pour tester chaque ville)

In [12]:
print("\n--- VALIDATION DES NOMS DE VILLES ---")

def valider_villes_dataframe(df, colonne_ville='ville'):
    villes_uniques = df[colonne_ville].dropna().unique()

    villes_invalides = []
    villes_valides = []

    for ville in villes_uniques:
        url = f"https://geo.api.gouv.fr/communes?nom={ville}&limit=1"
        try:
            response = requests.get(url, timeout=5)
            if response.status_code == 200 and len(response.json()) > 0:
                villes_valides.append(ville)
            else:
                villes_invalides.append(ville)
        except:
            villes_invalides.append(ville)

    print(f"Villes valides : {len(villes_valides)} soit {len(villes_valides)/len(villes_uniques)*100:.0f}%")
    print(f"Villes invalides : {len(villes_invalides)} soit {len(villes_invalides)/len(villes_uniques)*100:.0f}%")

    if villes_invalides:
        # Afficher les lignes concernées
        mask_invalides = df[colonne_ville].isin(villes_invalides)
        print(f"\nNombre de lignes affectées : {mask_invalides.sum()} soit {mask_invalides.sum()/len(df_clients)*100:.0f}%")
        display(df[mask_invalides][[colonne_ville]].head(10))

    return villes_valides, villes_invalides

villes_valides, villes_invalides = valider_villes_dataframe(df_clients)



--- VALIDATION DES NOMS DE VILLES ---
Villes valides : 87 soit 24%
Villes invalides : 272 soit 76%

Nombre de lignes affectées : 317 soit 61%


,ville
0,Lejeune
2,Lopezdan
4,Sainte Jacques-les-Bains
6,Petit-sur-Mer
14,Sainte Lucieboeuf
15,Augernec
16,Pichon-sur-Mer
17,Blanc-sur-Mer
19,Saint ValentineVille
22,Saint Thomasdan
